import dataset

In [1]:
import pandas as pd 

housing_advert = pd.read_csv('housing-Raoofi.csv')

C:\Users\M7600QE\AppData\Local\Temp\ipykernel_24388\2671681662.py:3: DtypeWarning: Columns (0,1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  housing_advert = pd.read_csv('housing-Raoofi.csv')


data preprocessing

In [2]:
housing_advert.sample(5)

,Area,Construction,Room,Warehouse,Parking,Elevator,Address,Price
28853,50,1397,1,True,True,False,۳ روز پیش در تهران، جیحون,۲٬۹۲۰٬۰۰۰٬۰۰۰ تومان
26049,200,1401,3,True,True,True,۲ روز پیش در تهران، مبارک‌آباد بهشتی,۶٬۰۰۰٬۰۰۰٬۰۰۰ تومان
34056,60,1398,1,True,True,True,۳ روز پیش در تهران، کوی فردوس,۵٬۸۰۰٬۰۰۰٬۰۰۰ تومان
80211,90,1401,2,True,False,True,۱ هفته پیش در تهران، سبلان,۵٬۱۳۰٬۰۰۰٬۰۰۰ تومان
92313,120,1401,2,True,True,True,۱ روز پیش در تهران، پونک,۱۲٬۰۰۰٬۰۰۰٬۰۰۰ تومان


In [3]:
housing_advert.shape

(94036, 8)

In [4]:
housing_advert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94036 entries, 0 to 94035
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Area          94036 non-null  object
 1   Construction  94036 non-null  object
 2   Room          94036 non-null  object
 3   Warehouse     94036 non-null  object
 4   Parking       94036 non-null  object
 5   Elevator      94036 non-null  object
 6   Address       94036 non-null  object
 7   Price         94036 non-null  object
dtypes: object(8)
memory usage: 5.7+ MB


In [41]:
df = housing_advert.copy()

In [42]:
df['Area'] = pd.to_numeric(df['Area'], errors='coerce')
df['Construction'] = pd.to_numeric(df['Construction'], errors='coerce')
df['Room'] = pd.to_numeric(df['Room'], errors='coerce')

In [43]:
df['Warehouse'] = df['Warehouse'].astype(bool)
df['Parking'] = df['Parking'].astype(bool)
df['Elevator'] = df['Elevator'].astype(bool)

In [44]:
import re
def clean_address(address):
    match = re.search(r'پیش در تهران، ([^،]+)', address)
    if match:
        return match.group(1)
    return address

In [45]:
df['Address'] =  df['Address'].apply(clean_address)

In [46]:
def clean_price(price):
    cleaned_price = re.sub(r'[^\d]', '', price)
    return int(cleaned_price) if cleaned_price else None

In [47]:
df['Price'] = df['Price'].apply(clean_price)

In [48]:
df.isna().sum()

Area              1
Construction      1
Room              1
Warehouse         0
Parking           0
Elevator          0
Address           0
Price           246
dtype: int64

In [49]:
df.dropna(inplace=True)

In [50]:
df.describe()

,Area,Construction,Room,Price
count,93790.000000,93790.000000,93790.000000,9.379000e+04
mean,125.018083,1392.983463,2.225301,1.452574e+10
std,267.611046,7.745195,0.765839,2.737502e+10
min,1.000000,1371.000000,1.000000,2.300000e+08
25%,74.000000,1386.000000,2.000000,4.900000e+09
50%,105.000000,1395.000000,2.000000,8.580000e+09
75%,141.000000,1401.000000,3.000000,1.550000e+10
max,36500.000000,1401.000000,4.000000,1.000000e+12


In [ ]:
df['AddressCode'], unique_addresses = pd.factorize(df['Address'])

,Area,Construction,Room,Warehouse,Parking,Elevator,Address,Price,AddressCode
0,153.0,1393.0,3.0,True,True,True,درکه,1.836000e+10,0
1,418.0,1382.0,4.0,True,True,True,زعفرانیه,1.300000e+11,1
2,57.0,1385.0,2.0,True,True,True,جابری,2.600000e+09,2
3,193.0,1371.0,3.0,True,True,True,فرمانیه,2.300000e+10,3
4,62.0,1401.0,1.0,True,True,True,دهکده المپیک,1.410000e+09,4
...,...,...,...,...,...,...,...,...,...
94031,181.0,1393.0,3.0,True,True,True,جردن,1.800000e+10,59
94032,121.0,1400.0,3.0,True,True,True,اباذر,1.380000e+10,52
94033,140.0,1400.0,3.0,True,True,True,آبشار تهران,9.860000e+09,218
94034,105.0,1398.0,2.0,True,True,True,اوقاف,6.430000e+09,232


In [52]:
df[['Parking','Warehouse','Elevator']] = df[['Parking','Warehouse','Elevator']].astype(int)

thanks mohamadreza zaheri for this plot

https://www.kaggle.com/code/mohamadrezazaheri/divar-house-price

In [55]:
ads_per_neighborhood = df['Address'].value_counts().reset_index()
ads_per_neighborhood.columns = ['Neighborhood', 'Ads_Count']

In [56]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="tehran_ads_map")
ads_per_neighborhood['Coordinates'] = ads_per_neighborhood['Neighborhood'].apply(
    lambda x: geolocator.geocode(f"تهران, {x}", timeout=10) if pd.notnull(x) else None)
ads_per_neighborhood['Latitude'] = ads_per_neighborhood['Coordinates'].apply(lambda x: x.latitude if x else None)
ads_per_neighborhood['Longitude'] = ads_per_neighborhood['Coordinates'].apply(lambda x: x.longitude if x else None)

In [58]:
import folium

tehran_map = folium.Map(location=[35.6892, 51.3890], zoom_start=12 )
for idx, row in ads_per_neighborhood.iterrows():
    if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude']):
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=row['Ads_Count'] * 0.005, color='darkblue',fill=True,fill_color='darkred',fill_opacity=0.6,
            popup=f"محله: {row['Neighborhood']}, تعداد آگهی‌ها: {row['Ads_Count']}"
        ).add_to(tehran_map)

display(tehran_map)